<a href="https://colab.research.google.com/github/nin-ed/Neural-Network-Algorithms/blob/hacktoberfest/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torchvision import models, transforms, datasets
import torch
from torch import nn, optim
from zipfile import ZipFile

In [ ]:
with ZipFile('/content/drive/My Drive/Cat_Dog_data.zip', 'r') as f:
  print("Extracting...")
  f.extractall()
  print("Done")

Extracting...
Done


In [ ]:
train_transform = transforms.Compose([transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

test_transform = transforms.Compose([transforms.RandomRotation(30),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [ ]:
def Network(in_s, ou_s, hid, drop_p=0.2):
    modules = []
    modules.append(nn.Linear(in_s, hid[0]))
    modules.append(nn.ReLU())
    modules.append(nn.Dropout(p=drop_p))
    for i in range(1,len(hid)):
        modules.append(nn.Linear(hid[i-1], hid[i]))
        modules.append(nn.ReLU())
        modules.append(nn.Dropout(p=drop_p))
    modules.append(nn.Linear(hid[-1], ou_s))
    modules.append(nn.LogSoftmax(dim=1))
    model = nn.Sequential(*modules)
    return model

In [ ]:
def train(model, criterion, optimizer, trainloader, testloader, epochs, print_every):
    train_losses, test_losses = [], []
    for e in range(epochs):
        train_loss = 0
        for images, labels in trainloader:
            #images = images.view(images.shape[0], -1)
            images, labels = images.cuda(), labels.cuda()
            optimizer.zero_grad()
            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        test_loss, accuracy = validate(model, criterion, testloader)
        train_losses.append(train_loss / len(trainloader))
        test_losses.append(test_loss / len(testloader))
        model.train()

        if (e+1)%(print_every) == 0:
            print(f"----- Epoch {e+1} -----")
            print(f"Training Loss: {train_loss/len(trainloader)}")
            print(f"Testing Loss: {test_loss / len(testloader)}")
            print(f"Accuracy: {accuracy/len(testloader)}")

    torch.save(model.state_dict(), '/content/drive/My Drive/trl.pt')

    return train_losses, test_losses

In [ ]:
def validate(model, criterion, testloader):
    accuracy = 0
    test_loss = 0
    with torch.no_grad():
        for images, labels in testloader:
            #images = images.view(images.shape[0], -1)
            images, labels = images.cuda(), labels.cuda()
            output = model.forward(images)
            loss = criterion(output, labels)
            prob = torch.exp(output)
            top_prob, top_class = prob.topk(1, dim=1)
            equal = top_class == labels.view(*top_class.shape)
            test_loss += loss.item()
            accuracy += torch.mean(equal.type(torch.FloatTensor))

    return test_loss, accuracy

In [ ]:
train_data = datasets.ImageFolder('/content/Cat_Dog_data/train', transform=train_transform)
test_data = datasets.ImageFolder('/content/Cat_Dog_data/test', transform=test_transform)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)
model = models.densenet121(pretrained=True)
for par in model.parameters():
  par.requires_grad = False

model.classifier = Network(1024, 2, [512, 128])
model.classifier = model.classifier.cuda()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

In [ ]:
train(model, criterion, optimizer, trainloader, testloader, epochs=5, print_every=1)

----- Epoch 1 -----
Training Loss: 0.19998923711351713
Testing Loss: 0.09366109278053045
Accuracy: 0.963671863079071
----- Epoch 2 -----
Training Loss: 0.15959029117683796
Testing Loss: 0.08828865165123716
Accuracy: 0.9652343988418579
----- Epoch 3 -----
Training Loss: 0.14766563604247163
Testing Loss: 0.08671727925539016
Accuracy: 0.967578113079071
----- Epoch 4 -----
Training Loss: 0.1386322686237029
Testing Loss: 0.10929568593855947
Accuracy: 0.958203136920929
----- Epoch 5 -----
Training Loss: 0.14204392504391514
Testing Loss: 0.0760832099678737
Accuracy: 0.9683593511581421


([0.19998923711351713,
  0.15959029117683796,
  0.14766563604247163,
  0.1386322686237029,
  0.14204392504391514],
 [0.09366109278053045,
  0.08828865165123716,
  0.08671727925539016,
  0.10929568593855947,
  0.0760832099678737])